# Comparing different levels of regionalization

Your task is to see how our overall results change as our degree of regionalization changes. specifically, you need to look at the land (0) and water (1) impacts of ethanol from sugarcane (2) and maize (3).

(0) `('LC-IMPACT', 'Land Use', 'Occupation', 'Marginal', 'Certain')`

(1) `('LC-IMPACT', 'Water Use', 'Ecosystem Quality', 'Surface Water', 'Marginal', 'All')`

(2) `('Fuel comparison', 'sugarcane transport')`

(3) `('Fuel comparison', 'maize transport')`

In [ ]:
import bw2calc as bc
import bw2data as bd
import bw2regional as bwr
import bw2analyzer as ba
import csv
import geopandas as gpd
import pandas as pd
import numpy as np

In [ ]:
bd.projects.set_current("LC IMPACT case study")

To get the two-scale calculations working, we can hack the system:

In [ ]:
for gc in ['ecoinvent', 'RoW', 'world']:
    needed = (gc, 'world')
    if needed not in bwr.intersections:
        bwr.create_empty_intersection(needed)

In [ ]:
ca = ba.ContributionAnalysis()

## Site-generic LCA

The LCA class is `bc.LCA`. You only need to give a demand and method.

You can use `ca.annotated_top_processes()` and `ca.annotated_top_emissions()` to see the most important activities and elementary flows. Use the option `Show Contextual Help` (right-click) to see how to call these functions.

You can also use the function `ba.print_recursive_calculation` to explore the supply chain. Play with the different options :)

This is not a test, but you should uunderstand not just that these different LCA methods will produce different results, but also **why**.

In [ ]:
do

In [ ]:
some

In [ ]:
stuff

In [ ]:
here

In [ ]:
😊

# Shared Spatial Scale

Now do the calculations for regionalization with a shared spatial scale. We won't be able to enter data for ecoinvent-specific places like `foo without bar` or `RoW 147628`, but you can get the CFs on a country level. The data is here: https://lc-impact.eu/. You have a choice - you can load an entire worksheet into Pandas, and write every country CF, or you can type in the CFs for the most important countries (the contribution analysis from the site-generic results can tell you approximately which countries are important). Similarly, contribution analysis will tell you the most important bioisphere flows - you only need to add CFs for 3 or 4.

You can look at the `New LCIA Method` notebook - the format to write a new method (you will need to write a **new method**) is:

```
method_object.write([
    (key of biosphere flow, CF amount, location),
])
```

The key is something like `('biosphere3', 'sdfklr8kldfjsld')`, the CF amount is a number, and the location will be an ISO country code like `'ES'`. You can get the flow key from a flow object with `flow.key`.

The LCA class for a shared scale is `bwr.OneSpatialScaleLCA`.

# Two Spatial Scales

Do the same things (both demands and both methods - maybe in a loop instead of manually entering them) for the class `bwr.TwoSpatialScalesLCA`. The spatial intersections should be there already, this one is relatively easy.

Use LCA methods like `geodataframe_ia_spatial_scale` and `geodataframe_inv_spatial_scale` to get tables of results; you can then use `.explore` to visualize these maps. You can also use `.to_file('some_name.gpkg')` to create files you can download and visualize using QGIS.

# Two Spatial Scales with Loading

We don't have `loading` maps... you will have to make them up! Load the data from the LCIA method, and construct an artificial function that (maybe, up to you) give exponentially higher weights to higher CFs. See if you can also add some random noise.

To create a loading:

```
bwr.Loading(method name).write([
    (loading factor, location),
])
```

Where `method_name` is something like `('LC-IMPACT', 'Land Use', 'Occupation', 'Marginal', 'Certain')`, and locations are the same as in the LCIA factors.

# Extensions tables - did you miss them?

The last step. Here, let's use two extension tables, one for agriculture (crop-specific) and one for other activity.

In [ ]:
irrigation = [x.id for x in bd.Database("ecoinvent") if 'irrigation' in x['name']]
crops = [x.id for x in bd.Database("ecoinvent") 
         if x['name'] in ('sugarcane production', 'maize grain production')]

In [ ]:
activities = [
    ('Fuel comparison', 'sugarcane transport'),
    ('Fuel comparison', 'maize transport'),
    ('Fuel comparison', 'petrol transport'),
]

lcimpact = {
 ("LC-IMPACT", "Climate Change", "Human Health", "Marginal", "Certain", "100 Years"): None,
 ("LC-IMPACT", "Climate Change", "Human Health", "Marginal", "All", "100 Years"): None,
 ("LC-IMPACT", "Climate Change", "Human Health", "Marginal", "Certain", "Infinite"): None,
 ("LC-IMPACT", "Climate Change", "Human Health", "Marginal", "All", "Infinite"): None,
 ("LC-IMPACT", "Climate Change", "Terrestrial Ecosystems", "Marginal", "Certain", "100 Years"): None,
 ("LC-IMPACT", "Climate Change", "Terrestrial Ecosystems",  "Marginal", "All", "100 Years"): None,
 ("LC-IMPACT", "Climate Change", "Terrestrial Ecosystems",  "Marginal", "Certain", "Infinite"): None,
 ("LC-IMPACT", "Climate Change", "Terrestrial Ecosystems",  "Marginal", "All", "Infinite"): None,
 ("LC-IMPACT", "Climate Change", "Aquatic Ecosystems", "Marginal", "All", "100 Years"): None,
 ("LC-IMPACT", "Climate Change", "Aquatic Ecosystems",  "Marginal", "All", "Infinite"): None, 
 ('LC-IMPACT', 'Land Use', 'Occupation', 'Marginal', 'Certain'): {
     'xt others': 'world-topo-ecoregions-weighted-pop-density',
     'activities': crops,
     'sugarcane': 'world-topo-ecoregions-sugarcane_landuse_intensity',
     'maize': 'world-topo-ecoregions-maize_landuse_intensity',
 },
 ('LC-IMPACT', 'Particulate Matter Formation', 'Marginal', 'All'): {
     'xt others': 'world-topo-particulate-matter-weighted-pop-density',
     'activities': [],
     'sugarcane': 'world-topo-particulate-matter-sugarcane_landuse_intensity',
     'maize': 'world-topo-particulate-matter-maize_landuse_intensity',
 },
 ('LC-IMPACT', 'Particulate Matter Formation', 'Marginal', 'Certain'): {
     'xt others': 'world-topo-particulate-matter-weighted-pop-density',
     'activities': [],
     'sugarcane': 'world-topo-particulate-matter-sugarcane_landuse_intensity',
     'maize': 'world-topo-particulate-matter-maize_landuse_intensity',
 },
 ('LC-IMPACT', 'Water Use', 'Ecosystem Quality', 'Surface Water', 'Marginal', 'All'): {
     'xt others': 'world-topo-watersheds-eq-sw-all-weighted-pop-density',
     'activities': irrigation + crops,
     'sugarcane': 'world-topo-watersheds-eq-sw-all-sugarcane_water_intensity',
     'maize': 'world-topo-watersheds-eq-sw-all-maize_water_intensity',
 },
 ('LC-IMPACT', 'Water Use', 'Ecosystem Quality', 'Surface Water', 'Marginal', 'Certain'): {
     'xt others': 'world-topo-watersheds-eq-sw-certain-weighted-pop-density',
     'activities': irrigation + crops,
     'sugarcane': 'world-topo-watersheds-eq-sw-certain-sugarcane_water_intensity',
     'maize': 'world-topo-watersheds-eq-sw-certain-maize_water_intensity',
 },
 ('LC-IMPACT', 'Water Use', 'Human Health', 'Marginal'): {
     'xt others': 'world-topo-watersheds-hh-weighted-pop-density',
     'activities': irrigation + crops,
     'sugarcane': 'world-topo-watersheds-hh-sugarcane_water_intensity',
     'maize': 'world-topo-watersheds-hh-maize_water_intensity',
 },
}

In [ ]:
xt_ag = bwr.ExtensionTablesLCA(
    func_unit,
    method,
    xtable=lcimpact[method][crop],
    limitations={
        'activities': lcimpact[method]['activities'],
    },
)
xt_ag.lci()
xt_ag.lcia()
xt_ag.score

In [ ]:
xt_others = bwr.ExtensionTablesLCA(
    func_unit,
    method,
    xtable=lcimpact[method]['xt others'],
    limitations={
        'activities': lcimpact[method]['activities'],
        'activities mode': 'exclude'
    }
)
xt_others.lci()
xt_others.lcia()
xt_others.score

In [ ]:
xt_ag.characterized_inventory += xt_others.characterized_inventory

Plot, interpret, etc. - but not on XT spatial scale, as we have combined two of them.